<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/casimir_train_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

# 1. True Casimir force (negative sign included)
def casimir_force(d, ε):
    return - (np.pi**2) / (240.0 * d**4) * ε

# 2. Create synthetic dataset
#    distances: 10 nm → 1 µm, permittivities: 1 → 10
n_dist, n_perm = 100, 100
distances = np.linspace(1e-8, 1e-6, n_dist)
permittivities = np.linspace(1.0, 10.0, n_perm)
D, E = np.meshgrid(distances, permittivities)
X = np.stack([D.ravel(), E.ravel()], axis=1)
y = casimir_force(X[:,0], X[:,1]).reshape(-1,1)

# 3. Convert to tensors and normalize
#    log-distance helps the network learn the 1/d^4 law
log_d = np.log(X[:,0])
log_d_norm = (log_d - log_d.mean()) / log_d.std()
ε_norm     = (X[:,1] - X[:,1].mean()) / X[:,1].std()

X_tensor = torch.tensor(np.stack([log_d_norm, ε_norm], axis=1), dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)
loader  = DataLoader(dataset, batch_size=256, shuffle=True)

# 4. Define the MLP with LayerNorm
class NegativeEnergyAI(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()
        layers = []
        last_dim = input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(last_dim, h),
                nn.LayerNorm(h),
                nn.ReLU(inplace=True)
            ]
            last_dim = h
        layers.append(nn.Linear(last_dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

model = NegativeEnergyAI(input_dim=2, hidden_dims=[64,64,32], output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# 5. Training loop
n_epochs = 200
for epoch in range(1, n_epochs+1):
    model.train()
    total_loss = 0.0
    for xb, yb in loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss  = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    total_loss /= len(dataset)
    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d}  MSE Loss: {total_loss:.3e}")

# 6. Evaluate on grid and plot
model.eval()
with torch.no_grad():
    preds = model(X_tensor).numpy().flatten()

# Plot predicted vs true force
plt.figure(figsize=(6,6))
plt.scatter(y, preds, s=5, alpha=0.3)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel("True Casimir Force")
plt.ylabel("Predicted Force")
plt.title("Model Fit")
plt.tight_layout()
plt.show()

# Surface visualization (heatmap)
Z_true = y.reshape(n_perm, n_dist)
Z_pred = preds.reshape(n_perm, n_dist)

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,4))
im1 = ax1.pcolormesh(distances*1e6, permittivities, Z_true, shading='auto')
ax1.set_title("True Force Surface")
ax1.set_xlabel("Distance (µm)")
ax1.set_ylabel("Permittivity")
fig.colorbar(im1, ax=ax1)

im2 = ax2.pcolormesh(distances*1e6, permittivities, Z_pred, shading='auto')
ax2.set_title("Predicted Force Surface")
ax2.set_xlabel("Distance (µm)")
ax2.set_ylabel("Permittivity")
fig.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()